In [25]:
# coding:utf8
from PIL import Image, ImageDraw, ImageFont
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import numpy as np
import math
import csv
import os
import codecs

In [26]:
def srew(input_array, num):
    print(input_array.shape)
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)
    
    for x in range(width):
        for y in range(height):
            
            a = y*num
            i, d = divmod(a, 1)
            if i+x < width:
                output_array[y][x] = input_array[y][int(i+x)] *255   
    new_image = Image.fromarray(output_array.astype(np.uint8))
    new_image.save('NEW1.bmp')
    return output_array

    

In [27]:
def create_folder(dir, folder_name):

    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [28]:
def profile_y(input_array):
    output_array2 = input_array
    return {'y_axis' : np.arange(0, output_array2.shape[0]).astype(int), # разметка оси y, 0 тк первое значение размера это колво строк
            'values' : np.sum(output_array2, axis=1) # проекция на ось y
            }

def profile_x(input_array):
    output_array1 = input_array
    return {'values' : np.sum(output_array1, axis=0), # проекция на ось x
            'x_axis' : np.arange(0, output_array1.shape[1]).astype(int) # разметка оси x
            }

def X_diagram(input_array):
    
    height, width = input_array.shape[:2]
    p = profile_x(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    
    for x in range(width):
        for y in range(height):
            if y+1 <= p['values'][x]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    new_image = Image.fromarray(output_array.astype(np.uint8))
    # new_image.save('NEW0.bmp')
    return new_image

    
def Y_diagram(input_array):
    height, width = input_array.shape[:2]

    p = profile_y(input_array)
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for x in range(width):
        for y in range(height):
            if x+1 <= p['values'][y]:
                output_array[y][x] = 0
            else:
                output_array[y][x] = 255

    new_image = Image.fromarray(output_array.astype(np.uint8))
    # new_image.save('NEW1.bmp')
    return new_image


In [29]:
def monochrome(input_array):
    # print(np.shape(input_array)[:])
    height, width = input_array.shape[:2]
    output_array = np.zeros((height, width), dtype=input_array.dtype)

    for y in range(height):
        for x in range(width):
            output_array[y][x] = np.mean(input_array[y][x])

    return output_array
  
def binarization_b1w0(input_array):
    height, width = input_array.shape[:2]
    input_array1 = monochrome(input_array)
    # print(np.shape(input_array)[:])
    output_array = np.zeros((height, width), int)

    for i in range(height):
        for j in range(width):
            if input_array1[i][j] <= 200:
                output_array[i][j] = 1
            else:
                output_array[i][j] = 0
    
    return output_array

def first_nonzero(arr):
    first_non0 = -1
    for i in range (np.shape(arr)[0]):
        if arr[i] > 0:
            first_non0 = i
            break

    return first_non0

def last_nonzero(arr):
    arr_reversed = np.flip(arr) 
    first_non0 = -1

    for i in range (np.shape(arr)[0]):
        if arr_reversed[i] > 0:
            first_non0 = np.shape(arr)[0] - i #!!!!!!!!!!!!!!!!!!
            break
    return first_non0

    
def edit_img(img):
    img_array = np.array(img)

    img_array_01 = binarization_b1w0(img_array)
    # angle = np.pi / 8
    img_array_cut = img_array_01[first_nonzero( profile_y(img_array_01)['values'] ) : last_nonzero( profile_y(img_array_01)['values'] ),
                                 first_nonzero( profile_x(img_array_01)['values'] ) : last_nonzero( profile_x(img_array_01)['values'] )]

    return img_array_cut

In [30]:
def XY_diagram_join(input_image, path):

    image_array = np.array(input_image)
    image_array_mono = monochrome(image_array)
    aaa = binarization_b1w0(image_array_mono)

    y = Y_diagram(aaa)
    x = X_diagram(aaa)

    height, width = image_array_mono.shape[:2]

    X_array = np.array(x)
    Y_array = np.array(y)
    zero_array  = np.ones((height,width),  dtype=image_array_mono.dtype)*255

#     print(np.shape(image_array_mono),
#           np.shape(X_array),
#           np.shape(Y_array),
#           np.shape(zero_array)
#           )
    final0 = np.vstack([Y_array, zero_array])
    final1 = np.vstack([image_array_mono, X_array])
    final = np.hstack([final1, final0])   

    new_image = Image.fromarray(final.astype(np.uint8))
    new_image.save(os.path.join(path, "diagram.bmp"))

In [31]:
def create_folder(dir, folder_name):

    output_dir = os.path.join(dir, folder_name)

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [32]:
def get_characteristics(img_edited):

    # Вес (масса чёрного) каждой четверти изображения символа
    weight = img_edited.sum()

    # Удельный вес (вес, нормированный к четверти площади)
    rel_weight = weight / (img_edited.shape[0] * img_edited.shape[1])

    # Координаты центра тяжести
    x_avg = 0
    for x, column in enumerate(img_edited.T):
        x_avg += sum((x + 1) * column)
    x_avg = x_avg / weight

    y_avg = 0
    for y, row in enumerate(img_edited):
        y_avg += sum((y + 1) * row)
    y_avg = y_avg / weight
    
    # Нормированные координаты центра тяжести
    rel_x_avg = (x_avg - 1) / (img_edited.shape[1] - 1)
    rel_y_avg = (y_avg - 1) / (img_edited.shape[0] - 1)

    # Осевые моменты инерции по горизонтали и вертикали
    iner_x = 0
    for y, row in enumerate(img_edited):
        iner_x += sum((y + 1 - y_avg) * 2 * row)

    iner_y = 0
    for x, column in enumerate(img_edited.T):
        iner_y += sum((x + 1 - x_avg) * 2 * column)

    # Нормированные осевые моменты инерции
    rel_iner_x = iner_x / (img_edited.shape[0] * 2 + img_edited.shape[1] * 2)
    rel_iner_y = iner_y / (img_edited.shape[0] * 2 + img_edited.shape[1] * 2)

    return {
        'weight': weight,
        'rel_weight': rel_weight,
        'center': [x_avg, y_avg],
        'rel_center': [rel_x_avg, rel_y_avg],
        'inertia': [iner_x, iner_y],
        'rel_inertia': [rel_iner_x, rel_iner_y]
    }

In [33]:
def errosion(input_array):

    height, width = input_array.shape[:2]
    output_array = input_array.copy()
    for i in range (1, width-1):
        for j in range (1, height-1):
            if input_array[j][i] == 1 and (input_array[j][i+1] == 0 or input_array[j][i-1] == 0):
                output_array[j][i] = 0
            else:
                output_array[j][i] = input_array[j][i]

    return output_array

In [34]:
def segmentation(input_array1):

    height, width = input_array1.shape[:2]
    input_array = errosion(input_array1)
    # input_array= errosion(input_array2)
    # angle = np.pi / 8
    p = profile_x(input_array)
    list_borders = {
        'left' : [],
        'right' : [],
        'up': [],
        'down' : []
    }

    for i in range (p['x_axis'].shape[0]-1) :  
        if (p['values'][i] > 0) and (p['values'][i+1] == 0):
            list_borders['right'].append(i)
        if (p['values'][i] ==  0) and (p['values'][i+1] > 0):
            list_borders['left'].append(i)

    if list_borders['left'][0]>list_borders['right'][0]:
        list_borders['left'].append(0)
    if list_borders['left'][-1]<list_borders['right'][-1]:
        list_borders['right'].append(width-1)

    list_borders['left'].sort()
    list_borders['right'].sort()

    for k in range (len(list_borders['left'])):

        if list_borders['left'][k]<list_borders['right'][k]:

            new = np.zeros((height, list_borders['right'][k]-list_borders['left'][k]), int)
            height, width = new.shape[:2]
            for i in range (width):
                for j in range (height):
                    new[j][i]= input_array[j][list_borders['left'][k]+i]
            py = profile_y(new)
            list_borders['up'].append(first_nonzero(py['values']))
            list_borders['down'].append(last_nonzero(py['values']))


    return list_borders

def draw_brders(input_image, list, path):

    img_array = np.array(input_image)
    binb1w0 = binarization_b1w0(img_array)
    # angle = np.pi / 8
    height, width = img_array.shape[:2]
    y = profile_y(binb1w0)
    x = profile_x(binb1w0)

    y_0 = first_nonzero(y['values'])
    x_0 = first_nonzero(x['values'])

    for k in range (len(list['left'])):
        x1 = list['left'][k]
        x2 = list['right'][k]
        y1 = list['up'][k]
        y2 = list['down'][k]
        

        for i in range (width):
            for j in range  (height):
                if (j >= y1 and j <=y2 ) and (i >= x1 and i <= x2):
                    
                    if (j == y1 or j == y2 or i == x1 or i ==x2):
                        img_array[y_0+j][x_0+i][0] = 255
                        img_array[y_0+j][x_0+i][1] = 0
                        img_array[y_0+j][x_0+i][2] = 0


    new_image = Image.fromarray(img_array)
    new_image.save(os.path.join(path, "borders.bmp"))

In [35]:
def save_profiles_as_graphics(img, path):
    x = profile_x(img)
    y = profile_y(img)

    for axis in x.items():
        plt.bar(x['x_axis'], x['values'], color='blue')
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile__axis_x.png'))  # Сохранение диаграммы как изображения bmp
        plt.close()  # Очистка

    for axis in y.items():
        plt.barh(y['y_axis'], y['values'], color='blue') # Используйте профили для создания столбчатой диаграммы
        plt.xlabel('Сумма значений')  # Подпись оси X
        plt.ylabel('Пиксели')  # Подпись оси Y
        plt.savefig(os.path.join( path, f'profile__axis_y.png'))  # Сохранение диаграммы как изображения bmp
        plt.close()  # Очистка


In [36]:
def main():
 
    DIR_cwd = os.getcwd()
    DIR_output = create_folder(DIR_cwd, "lab6 output")
    angle = np.pi / 8 
    input_image = Image.open('test1.bmp')
    input_array = np.array(input_image)
    print(input_array.shape[:])

    aaaa = edit_img(input_array)
    # aaaa1 = srew(aaaa, -0.2)
    X_diagram(aaaa)
    Y_diagram(aaaa)
    list = segmentation(aaaa)
    draw_brders(input_image , list, DIR_output)

    # input_image = Image.open('text3.bmp')
    # input_array = np.array(input_image)
    # XY_diagram_join(input_array, DIR_output)
    # save_profiles_as_graphics(aaaa,DIR_output)



if __name__ == '__main__':
    main()

(180, 1303, 3)
